In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.cluster import KMeans
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
bs_ex_matrix = pd.read_csv('expression_matrix.csv', header = None,  index_col=[0])
bs_ex_matrix.head()
bs_ex_matrix.shape

(16151, 524)

In [3]:
bs_col_data = pd.read_csv('columns_metadata.csv', header = 0,  index_col=[0])
bs_col_data.head(20)

,donor_id,donor_name,age,gender,structure_id,structure_acronym,structure_name
column_num,,,,,,,
1,13058,H376.IIA.51,8 pcw,M,10268,Ocx,occipital neocortex
2,13058,H376.IIA.51,8 pcw,M,10291,M1C-S1C,primary motor-sensory cortex (samples)
3,13058,H376.IIA.51,8 pcw,M,10361,AMY,amygdaloid complex
4,13058,H376.IIA.51,8 pcw,M,10550,MGE,medial ganglionic eminence
5,13058,H376.IIA.51,8 pcw,M,10243,STC,posterior (caudal) superior temporal cortex (area 22c)
6,13058,H376.IIA.51,8 pcw,M,10665,URL,upper (rostral) rhombic lip
7,13058,H376.IIA.51,8 pcw,M,10552,CGE,caudal ganglionic eminence
8,13058,H376.IIA.51,8 pcw,M,10391,DTH,dorsal thalamus
9,13058,H376.IIA.51,8 pcw,M,10278,MFC,anterior (rostral) cingulate (medial prefrontal) cortex


In [4]:
bs_row_data = pd.read_csv('rows_metadata.csv', header = 0,  index_col=[0])
bs_row_data.head()

,gene_id,ensembl_gene_id,gene_symbol,entrez_id
row_num,,,,
1,7062.0,ENSG00000000003,TSPAN6,7105.0
2,40735.0,ENSG00000000005,TNMD,64102.0
3,8736.0,ENSG00000000419,DPM1,8813.0
4,36423.0,ENSG00000000457,SCYL3,57147.0
5,35021.0,ENSG00000000460,C1orf112,55732.0


In [5]:
#I'm going to just cut the dataframe off at 16151 rows and hope that's correct. There is nothing in the readme that came with the files
#that explains why there are more genes than in the expression matrix
row_data_clean = bs_row_data.iloc[0:16151,:]
row_data_clean.tail()

,gene_id,ensembl_gene_id,gene_symbol,entrez_id
row_num,,,,
16147,123833.0,ENSG00000184954,OR6C70,390327.0
16148,4562.0,ENSG00000184956,MUC6,4588.0
16149,NaN,ENSG00000184961,AL772307.1,NaN
16150,54890.0,ENSG00000184967,NOC4L,79050.0
16151,11113.0,ENSG00000184979,USP18,11274.0


In [6]:
genes = row_data_clean['ensembl_gene_id']
len(genes)

16151

In [7]:
structures = bs_col_data['structure_name']
ex_clean = bs_ex_matrix.copy()
ex_clean.columns = structures
ex_clean.reset_index(inplace=True)
ex_clean = ex_clean.drop(columns = 0)
ex_clean.head()

structure_name,occipital neocortex,primary motor-sensory cortex (samples),amygdaloid complex,medial ganglionic eminence,posterior (caudal) superior temporal cortex (area 22c),upper (rostral) rhombic lip,caudal ganglionic eminence,dorsal thalamus,anterior (rostral) cingulate (medial prefrontal) cortex,dorsolateral prefrontal cortex,orbital frontal cortex,lateral ganglionic eminence,"inferolateral temporal cortex (area TEv, area 20)",hippocampus (hippocampal formation),ventrolateral prefrontal cortex,parietal neocortex,dorsolateral prefrontal cortex,anterior (rostral) cingulate (medial prefrontal) cortex,amygdaloid complex,dorsal thalamus,upper (rostral) rhombic lip,lateral ganglionic eminence,primary motor-sensory cortex (samples),medial ganglionic eminence,temporal neocortex,hippocampus (hippocampal formation),occipital neocortex,caudal ganglionic eminence,orbital frontal cortex,parietal neocortex,hippocampus (hippocampal formation),dorsal thalamus,amygdaloid complex,primary auditory cortex (core),"primary visual cortex (striate cortex, area V1/17)",anterior (rostral) cingulate (medial prefrontal) cortex,striatum,"primary motor cortex (area M1, area 4)",dorsolateral prefrontal cortex,"inferolateral temporal cortex (area TEv, area 20)",ventrolateral prefrontal cortex,posterior (caudal) superior temporal cortex (area 22c),posteroventral (inferior) parietal cortex,orbital frontal cortex,"primary somatosensory cortex (area S1, areas 3,1,2)",posterior (caudal) superior temporal cortex (area 22c),"primary visual cortex (striate cortex, area V1/17)",primary auditory cortex (core),cerebellum,"inferolateral temporal cortex (area TEv, area 20)",posteroventral (inferior) parietal cortex,"primary somatosensory cortex (area S1, areas 3,1,2)",dorsal thalamus,hippocampus (hippocampal formation),dorsolateral prefrontal cortex,orbital frontal cortex,"primary motor cortex (area M1, area 4)",ventrolateral prefrontal cortex,amygdaloid complex,striatum,amygdaloid complex,"primary motor cortex (area M1, area 4)","inferolateral temporal cortex (area TEv, area 20)",ventrolateral prefrontal cortex,striatum,cerebellar cortex,dorsal thalamus,dorsolateral prefrontal cortex,orbital frontal cortex,"primary somatosensory cortex (area S1, areas 3,1,2)","primary visual cortex (striate cortex, area V1/17)",anterior (rostral) cingulate (medial prefrontal) cortex,hippocampus (hippocampal formation),posteroventral (inferior) parietal cortex,primary auditory cortex (core),striatum,"primary motor cortex (area M1, area 4)",amygdaloid complex,"primary somatosensory cortex (area S1, areas 3,1,2)","primary visual cortex (striate cortex, area V1/17)",primary auditory cortex (core),hippocampus (hippocampal formation),ventrolateral prefrontal cortex,anterior (rostral) cingulate (medial prefrontal) cortex,posterior (caudal) superior temporal cortex (area 22c),orbital frontal cortex,posteroventral (inferior) parietal cortex,dorsolateral prefrontal cortex,"inferolateral temporal cortex (area TEv, area 20)",striatum,"primary somatosensory cortex (area S1, areas 3,1,2)","primary visual cortex (striate cortex, area V1/17)",amygdaloid complex,posteroventral (inferior) parietal cortex,cerebellum,dorsolateral prefrontal cortex,orbital frontal cortex,posterior (caudal) superior temporal cortex (area 22c),primary auditory cortex (core),hippocampus (hippocampal formation),"primary motor cortex (area M1, area 4)","inferolateral temporal cortex (area TEv, area 20)",mediodorsal nucleus of thalamus,anterior (rostral) cingulate (medial prefrontal) cortex,ventrolateral prefrontal cortex,"primary somatosensory cortex (area S1, areas 3,1,2)",dorsolateral prefrontal cortex,cerebellum,amygdaloid complex,"inferolateral temporal cortex (area TEv, area 20)",hippocampus (hippocampal formation),posteroventral (inferior) parietal cortex,primary auditory cortex (core),ventrolateral prefrontal cortex,striatum,orbital frontal cortex,anterior (rostral) cingulate (medial prefrontal) cortex,"primary visua

In [8]:
#I want to get this gene expression data into a transactional dataset so that I can do association rule mining on it. 
#What makes the most sense to me is to first break the dataset up by brain structure and treat each patient as a transaction
#I will discretize the data into different expression level categories based on the stats for each column

In [9]:
ex_clean.index = genes
ex_clean.head()

structure_name,occipital neocortex,primary motor-sensory cortex (samples),amygdaloid complex,medial ganglionic eminence,posterior (caudal) superior temporal cortex (area 22c),upper (rostral) rhombic lip,caudal ganglionic eminence,dorsal thalamus,anterior (rostral) cingulate (medial prefrontal) cortex,dorsolateral prefrontal cortex,orbital frontal cortex,lateral ganglionic eminence,"inferolateral temporal cortex (area TEv, area 20)",hippocampus (hippocampal formation),ventrolateral prefrontal cortex,parietal neocortex,dorsolateral prefrontal cortex,anterior (rostral) cingulate (medial prefrontal) cortex,amygdaloid complex,dorsal thalamus,upper (rostral) rhombic lip,lateral ganglionic eminence,primary motor-sensory cortex (samples),medial ganglionic eminence,temporal neocortex,hippocampus (hippocampal formation),occipital neocortex,caudal ganglionic eminence,orbital frontal cortex,parietal neocortex,hippocampus (hippocampal formation),dorsal thalamus,amygdaloid complex,primary auditory cortex (core),"primary visual cortex (striate cortex, area V1/17)",anterior (rostral) cingulate (medial prefrontal) cortex,striatum,"primary motor cortex (area M1, area 4)",dorsolateral prefrontal cortex,"inferolateral temporal cortex (area TEv, area 20)",ventrolateral prefrontal cortex,posterior (caudal) superior temporal cortex (area 22c),posteroventral (inferior) parietal cortex,orbital frontal cortex,"primary somatosensory cortex (area S1, areas 3,1,2)",posterior (caudal) superior temporal cortex (area 22c),"primary visual cortex (striate cortex, area V1/17)",primary auditory cortex (core),cerebellum,"inferolateral temporal cortex (area TEv, area 20)",posteroventral (inferior) parietal cortex,"primary somatosensory cortex (area S1, areas 3,1,2)",dorsal thalamus,hippocampus (hippocampal formation),dorsolateral prefrontal cortex,orbital frontal cortex,"primary motor cortex (area M1, area 4)",ventrolateral prefrontal cortex,amygdaloid complex,striatum,amygdaloid complex,"primary motor cortex (area M1, area 4)","inferolateral temporal cortex (area TEv, area 20)",ventrolateral prefrontal cortex,striatum,cerebellar cortex,dorsal thalamus,dorsolateral prefrontal cortex,orbital frontal cortex,"primary somatosensory cortex (area S1, areas 3,1,2)","primary visual cortex (striate cortex, area V1/17)",anterior (rostral) cingulate (medial prefrontal) cortex,hippocampus (hippocampal formation),posteroventral (inferior) parietal cortex,primary auditory cortex (core),striatum,"primary motor cortex (area M1, area 4)",amygdaloid complex,"primary somatosensory cortex (area S1, areas 3,1,2)","primary visual cortex (striate cortex, area V1/17)",primary auditory cortex (core),hippocampus (hippocampal formation),ventrolateral prefrontal cortex,anterior (rostral) cingulate (medial prefrontal) cortex,posterior (caudal) superior temporal cortex (area 22c),orbital frontal cortex,posteroventral (inferior) parietal cortex,dorsolateral prefrontal cortex,"inferolateral temporal cortex (area TEv, area 20)",striatum,"primary somatosensory cortex (area S1, areas 3,1,2)","primary visual cortex (striate cortex, area V1/17)",amygdaloid complex,posteroventral (inferior) parietal cortex,cerebellum,dorsolateral prefrontal cortex,orbital frontal cortex,posterior (caudal) superior temporal cortex (area 22c),primary auditory cortex (core),hippocampus (hippocampal formation),"primary motor cortex (area M1, area 4)","inferolateral temporal cortex (area TEv, area 20)",mediodorsal nucleus of thalamus,anterior (rostral) cingulate (medial prefrontal) cortex,ventrolateral prefrontal cortex,"primary somatosensory cortex (area S1, areas 3,1,2)",dorsolateral prefrontal cortex,cerebellum,amygdaloid complex,"inferolateral temporal cortex (area TEv, area 20)",hippocampus (hippocampal formation),posteroventral (inferior) parietal cortex,primary auditory cortex (core),ventrolateral prefrontal cortex,striatum,orbital frontal cortex,anterior (rostral) cingulate (medial prefrontal) cortex,"primary visua

In [54]:
donor_1 = ex_clean.iloc[:, :16]
donor_1 = donor_1.sample(2000)
donor_1.head(16)

structure_name,occipital neocortex,primary motor-sensory cortex (samples),amygdaloid complex,medial ganglionic eminence,posterior (caudal) superior temporal cortex (area 22c),upper (rostral) rhombic lip,caudal ganglionic eminence,dorsal thalamus,anterior (rostral) cingulate (medial prefrontal) cortex,dorsolateral prefrontal cortex,orbital frontal cortex,lateral ganglionic eminence,"inferolateral temporal cortex (area TEv, area 20)",hippocampus (hippocampal formation),ventrolateral prefrontal cortex,parietal neocortex
ensembl_gene_id,,,,,,,,,,,,,,,,
ENSG00000037241,11.920484,8.907539,14.694332,11.334771,8.669207,6.896212,10.339105,9.388867,12.549361,15.759559,15.278572,9.806136,9.663809,11.215255,12.871287,10.291769
ENSG00000176340,186.573724,116.163565,163.145834,194.119853,141.727315,140.760432,166.780160,196.191939,153.211797,201.518254,196.896667,157.452455,156.671678,179.422563,170.990864,151.408189
ENSG00000164161,0.011950,0.040927,0.047174,0.426715,0.056116,0.182720,0.198298,0.006759,0.032160,0.024341,0.185673,0.722693,0.146210,0.015750,0.099734,0.018364
ENSG00000091483,40.876719,35.039835,27.366198,27.392756,25.941200,21.400835,26.974930,24.704062,35.449702,43.696616,30.935220,27.433409,26.240347,29.540462,33.450545,36.507075
ENSG00000129562,115.122238,71.569545,79.307876,86.393232,81.904037,64.664726,78.326674,92.285293,118.220931,135.073078,145.980009,65.635070,83.917778,115.861777,102.905569,78.483544
ENSG00000124333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000164610,5.486399,6.511821,5.988585,6.727427,5.721388,5.050126,6.509979,3.862429,5.078091,6.148746,5.453755,6.101065,5.817562,5.032215,4.988197,5.912927
ENSG00000175806,3.547352,4.559174,3.110170,2.513691,3.730245,3.368179,2.836973,2.784848,3.928761,3.864946,3.293113,2.520750,3.294489,7.601210,4.139915,3.944346
ENSG00000133055,0.004570,0.020168,0.000000,0.330315,0.103833,0.019229,0.019275,0.000000,0.248221,0.027015,0.077429,0.131728,0.044372,0.062644,0.204888,0.181637


In [26]:
#make a new table with the stats
stats = pd.DataFrame(columns = donor_1.columns)
stats.head()


structure_name,occipital neocortex,primary motor-sensory cortex (samples),amygdaloid complex,medial ganglionic eminence,posterior (caudal) superior temporal cortex (area 22c),upper (rostral) rhombic lip,caudal ganglionic eminence,dorsal thalamus,anterior (rostral) cingulate (medial prefrontal) cortex,dorsolateral prefrontal cortex,orbital frontal cortex,lateral ganglionic eminence,"inferolateral temporal cortex (area TEv, area 20)",hippocampus (hippocampal formation),ventrolateral prefrontal cortex,parietal neocortex


In [27]:
#donor_1.iloc[0].mean()
means=[]
meds = []
mins =[]
maxs =[]
q1 = []
q2 =[]
q3 =[]

for i in range(0,len(donor_1.columns)):
    means.append(donor_1.iloc[i].mean())
    
stats.loc[len(stats)] = means

for i in range(0,len(donor_1.columns)):
    meds.append(donor_1.iloc[i].median())
    
stats.loc[len(stats)] = meds

for i in range(0,len(donor_1.columns)):
    mins.append(donor_1.iloc[i].min())
    
stats.loc[len(stats)] = mins

for i in range(0,len(donor_1.columns)):
    maxs.append(donor_1.iloc[i].max())
    
stats.loc[len(stats)] = maxs
stats.head()

structure_name,occipital neocortex,primary motor-sensory cortex (samples),amygdaloid complex,medial ganglionic eminence,posterior (caudal) superior temporal cortex (area 22c),upper (rostral) rhombic lip,caudal ganglionic eminence,dorsal thalamus,anterior (rostral) cingulate (medial prefrontal) cortex,dorsolateral prefrontal cortex,orbital frontal cortex,lateral ganglionic eminence,"inferolateral temporal cortex (area TEv, area 20)",hippocampus (hippocampal formation),ventrolateral prefrontal cortex,parietal neocortex
0,0.358017,0.0,0.00391,32.291686,0.003686,0.400457,528.700355,0.755795,1.256668,0.225489,9.882584,3.484886,0.001792,0.012334,1.171096,28.339484
1,0.324715,0.0,0.00028,31.849399,0.000000,0.337681,528.271816,0.436308,1.121626,0.189459,9.061423,3.488414,0.000000,0.011817,1.082203,28.485833
2,0.244631,0.0,0.00000,21.301316,0.000000,0.032651,234.410240,0.214669,0.954138,0.092805,2.534306,2.751094,0.000000,0.000000,0.603696,22.885390
3,0.676755,0.0,0.02759,45.357399,0.014507,1.178773,811.288269,2.649532,2.327572,0.458304,28.657847,4.617285,0.028673,0.039331,2.096822,40.336025


In [28]:
for i in range(0,len(donor_1.columns)):
    q1.append(donor_1.iloc[i].quantile(.25))
    
stats.loc[len(stats)] = q1

for i in range(0,len(donor_1.columns)):
    q2.append(donor_1.iloc[i].quantile(.5))
    
stats.loc[len(stats)] = q2

for i in range(0,len(donor_1.columns)):
    q3.append(donor_1.iloc[i].quantile(.75))
    
stats.loc[len(stats)] = q3

In [29]:
stats

structure_name,occipital neocortex,primary motor-sensory cortex (samples),amygdaloid complex,medial ganglionic eminence,posterior (caudal) superior temporal cortex (area 22c),upper (rostral) rhombic lip,caudal ganglionic eminence,dorsal thalamus,anterior (rostral) cingulate (medial prefrontal) cortex,dorsolateral prefrontal cortex,orbital frontal cortex,lateral ganglionic eminence,"inferolateral temporal cortex (area TEv, area 20)",hippocampus (hippocampal formation),ventrolateral prefrontal cortex,parietal neocortex
0,0.358017,0.0,0.00391,32.291686,0.003686,0.400457,528.700355,0.755795,1.256668,0.225489,9.882584,3.484886,0.001792,0.012334,1.171096,28.339484
1,0.324715,0.0,0.00028,31.849399,0.000000,0.337681,528.271816,0.436308,1.121626,0.189459,9.061423,3.488414,0.000000,0.011817,1.082203,28.485833
2,0.244631,0.0,0.00000,21.301316,0.000000,0.032651,234.410240,0.214669,0.954138,0.092805,2.534306,2.751094,0.000000,0.000000,0.603696,22.885390
3,0.676755,0.0,0.02759,45.357399,0.014507,1.178773,811.288269,2.649532,2.327572,0.458304,28.657847,4.617285,0.028673,0.039331,2.096822,40.336025
4,0.301801,0.0,0.00000,29.103148,0.000000,0.253007,408.923256,0.394344,1.028058,0.134613,5.951093,3.211654,0.000000,0.000000,0.943364,25.602250
5,0.324715,0.0,0.00028,31.849399,0.000000,0.337681,528.271816,0.436308,1.121626,0.189459,9.061423,3.488414,0.000000,0.011817,1.082203,28.485833
6,0.385794,0.0,0.00348,36.079992,0.007965,0.443409,626.886035,0.811302,1.453175,0.283151,12.884113,3.622047,0.000000,0.017780,1.405969,29.764677


In [30]:
stats.index = ['average', 'median', 'min', 'max', '1st quartile', '2nd quartile', '3rd quartile']
stats

structure_name,occipital neocortex,primary motor-sensory cortex (samples),amygdaloid complex,medial ganglionic eminence,posterior (caudal) superior temporal cortex (area 22c),upper (rostral) rhombic lip,caudal ganglionic eminence,dorsal thalamus,anterior (rostral) cingulate (medial prefrontal) cortex,dorsolateral prefrontal cortex,orbital frontal cortex,lateral ganglionic eminence,"inferolateral temporal cortex (area TEv, area 20)",hippocampus (hippocampal formation),ventrolateral prefrontal cortex,parietal neocortex
average,0.358017,0.0,0.00391,32.291686,0.003686,0.400457,528.700355,0.755795,1.256668,0.225489,9.882584,3.484886,0.001792,0.012334,1.171096,28.339484
median,0.324715,0.0,0.00028,31.849399,0.000000,0.337681,528.271816,0.436308,1.121626,0.189459,9.061423,3.488414,0.000000,0.011817,1.082203,28.485833
min,0.244631,0.0,0.00000,21.301316,0.000000,0.032651,234.410240,0.214669,0.954138,0.092805,2.534306,2.751094,0.000000,0.000000,0.603696,22.885390
max,0.676755,0.0,0.02759,45.357399,0.014507,1.178773,811.288269,2.649532,2.327572,0.458304,28.657847,4.617285,0.028673,0.039331,2.096822,40.336025
1st quartile,0.301801,0.0,0.00000,29.103148,0.000000,0.253007,408.923256,0.394344,1.028058,0.134613,5.951093,3.211654,0.000000,0.000000,0.943364,25.602250
2nd quartile,0.324715,0.0,0.00028,31.849399,0.000000,0.337681,528.271816,0.436308,1.121626,0.189459,9.061423,3.488414,0.000000,0.011817,1.082203,28.485833
3rd quartile,0.385794,0.0,0.00348,36.079992,0.007965,0.443409,626.886035,0.811302,1.453175,0.283151,12.884113,3.622047,0.000000,0.017780,1.405969,29.764677


In [31]:
zero_counts = (donor_1 == 0).sum(axis=0)
zero_counts

structure_name
occipital neocortex                                        280
primary motor-sensory cortex (samples)                     269
amygdaloid complex                                         250
medial ganglionic eminence                                 242
posterior (caudal) superior temporal cortex (area 22c)     257
upper (rostral) rhombic lip                                233
caudal ganglionic eminence                                 284
dorsal thalamus                                            292
anterior (rostral) cingulate (medial prefrontal) cortex    267
dorsolateral prefrontal cortex                             290
orbital frontal cortex                                     209
lateral ganglionic eminence                                254
inferolateral temporal cortex (area TEv, area 20)          250
hippocampus (hippocampal formation)                        249
ventrolateral prefrontal cortex                            268
parietal neocortex                      

In [34]:
#technically percent
zero_ratios = zero_counts/2000*100
zero_ratios

structure_name
occipital neocortex                                        14.00
primary motor-sensory cortex (samples)                     13.45
amygdaloid complex                                         12.50
medial ganglionic eminence                                 12.10
posterior (caudal) superior temporal cortex (area 22c)     12.85
upper (rostral) rhombic lip                                11.65
caudal ganglionic eminence                                 14.20
dorsal thalamus                                            14.60
anterior (rostral) cingulate (medial prefrontal) cortex    13.35
dorsolateral prefrontal cortex                             14.50
orbital frontal cortex                                     10.45
lateral ganglionic eminence                                12.70
inferolateral temporal cortex (area TEv, area 20)          12.50
hippocampus (hippocampal formation)                        12.45
ventrolateral prefrontal cortex                            13.40
parietal n

In [35]:
#If a gene is expressed above the 3rd quartile, it will be included in the dataset. (high expression only)
#Each structure will be a transaction and we will see if there are any association rules between the datasets
list0=[]
list1=[]
list2=[]
list3=[]
list4=[]
list5=[]
list6=[]
list7=[]
list8=[]
list9=[]
list10=[]
list11=[]
list12=[]
list13=[]
list14=[]
list15=[]

In [36]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,0] >= donor_1.iloc[0].quantile(.75)):
        list0.append(donor_1.index[j])
len(list0)

1385

In [37]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,1] >= donor_1.iloc[1].quantile(.75)):
        list1.append(donor_1.index[j])
len(list1)

2000

In [38]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,2] >= donor_1.iloc[2].quantile(.75)):
        list2.append(donor_1.index[j])
len(list2)

1750

In [39]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,3] >= donor_1.iloc[3].quantile(.75)):
        list3.append(donor_1.index[j])
len(list3)

146

In [40]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,4] >= donor_1.iloc[4].quantile(.75)):
        list4.append(donor_1.index[j])
len(list4)

1732

In [41]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,5] >= donor_1.iloc[5].quantile(.75)):
        list5.append(donor_1.index[j])
len(list5)

1424

In [42]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,6] >= donor_1.iloc[6].quantile(.75)):
        list6.append(donor_1.index[j])
len(list6)

1

In [43]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,7] >= donor_1.iloc[7].quantile(.75)):
        list7.append(donor_1.index[j])
len(list7)

1244

In [44]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,8] >= donor_1.iloc[8].quantile(.75)):
        list8.append(donor_1.index[j])
len(list8)

1164

In [45]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,9] >= donor_1.iloc[9].quantile(.75)):
        list9.append(donor_1.index[j])
len(list9)

1413

In [46]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,10] >= donor_1.iloc[10].quantile(.75)):
        list10.append(donor_1.index[j])
len(list10)

421

In [47]:
list11 = []
for j in range(len(donor_1)):
    if(donor_1.iloc[j,11] >= donor_1.iloc[11].quantile(.75)):
        list11.append(donor_1.index[j])
len(list11)

971

In [48]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,12] >= donor_1.iloc[12].quantile(.75)):
        list12.append(donor_1.index[j])
len(list12)

2000

In [49]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,13] >= donor_1.iloc[13].quantile(.75)):
        list13.append(donor_1.index[j])
len(list13)

1712

In [50]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,14] >= donor_1.iloc[14].quantile(.75)):
        list14.append(donor_1.index[j])
len(list14)

1175

In [51]:
for j in range(len(donor_1)):
    if(donor_1.iloc[j,15] >= donor_1.iloc[15].quantile(.75)):
        list15.append(donor_1.index[j])
len(list15)

177

In [52]:
g_trans =pd.DataFrame()
g_trans = pd.DataFrame([list0,list1,list2,list3,list4,list5,list6,list7,list8,list9,list10,list11,list12,list13,list14,list15])
g_trans.index = donor_1.columns
g_trans.iloc[:,:4]

,0,1,2,3
structure_name,,,,
occipital neocortex,ENSG00000108312,ENSG00000026025,ENSG00000112964,ENSG00000077454
primary motor-sensory cortex (samples),ENSG00000105808,ENSG00000179407,ENSG00000126838,ENSG00000108312
amygdaloid complex,ENSG00000105808,ENSG00000108312,ENSG00000146038,ENSG00000026025
medial ganglionic eminence,ENSG00000026025,ENSG00000110107,ENSG00000116288,ENSG00000170233
posterior (caudal) superior temporal cortex (area 22c),ENSG00000105808,ENSG00000126838,ENSG00000108312,ENSG00000160224
upper (rostral) rhombic lip,ENSG00000105808,ENSG00000108312,ENSG00000026025,ENSG00000112964
caudal ganglionic eminence,ENSG00000053438,None,None,None
dorsal thalamus,ENSG00000108312,ENSG00000026025,ENSG00000077454,ENSG00000134817
anterior (rostral) cingulate (medial prefrontal) cortex,ENSG00000108312,ENSG00000026025,ENSG00000134817,ENSG00000047188


In [53]:
g_trans.to_csv('Gene_Transactions_2000.csv', index=True)